In [1]:
import pygame
import sys
import copy

# ---------------------- Lógica do mundo ----------------------
directions = [(1, 0), (0, 1), (-1, 0), (0, -1)]

class Ambiente:
    def __init__(self, size=4):
        self.size = size
        self.pits = {(2, 2), (4, 2)}
        self.wumpus = (3, 1)
        self.gold = (2, 3)

    def dentro(self, x, y):
        return 1 <= x <= self.size and 1 <= y <= self.size

    def percept(self, x, y, wumpus_alive=True):
        p = {"stench": False, "breeze": False, "glitter": False}
        if (x, y) == self.gold:
            p["glitter"] = True
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if self.dentro(nx, ny):
                if (nx, ny) in self.pits:
                    p["breeze"] = True
                if wumpus_alive and (nx, ny) == self.wumpus:
                    p["stench"] = True
        return p


class BaseConhecimento:
    def __init__(self, size):
        self.size = size
        self.safe = set()
        self.poss_pit = set()
        self.poss_wumpus = set()
        self.visited = set()

    def adjacentes(self, x, y):
        adj = []
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if 1 <= nx <= self.size and 1 <= ny <= self.size:
                adj.append((nx, ny))
        return adj

    def atualizar(self, x, y, percept):
        self.visited.add((x, y))
        self.safe.add((x, y))

        adj = self.adjacentes(x, y)

        if not percept["breeze"]:
            for a in adj:
                self.safe.add(a)
                self.poss_pit.discard(a)
        else:
            for a in adj:
                if a not in self.safe:
                    self.poss_pit.add(a)

        if not percept["stench"]:
            for a in adj:
                self.safe.add(a)
                self.poss_wumpus.discard(a)
        else:
            for a in adj:
                if a not in self.safe:
                    self.poss_wumpus.add(a)

    def proxima_segura(self):
        seguras = self.safe - self.visited
        return next(iter(seguras)) if seguras else None


class AgenteLogico:
    def __init__(self, ambiente):
        self.env = ambiente
        self.kb = BaseConhecimento(ambiente.size)
        self.x, self.y = 1, 1
        self.has_gold = False
        self.wumpus_alive = True
        self.history = []
        self.percepts = []
        self.kb.safe.add((1,1))

    def save_state(self):
        st = {
            "x": self.x, "y": self.y, "has_gold": self.has_gold,
            "wumpus_alive": self.wumpus_alive,
            "visited": copy.deepcopy(self.kb.visited),
            "safe": copy.deepcopy(self.kb.safe),
            "poss_pit": copy.deepcopy(self.kb.poss_pit),
            "poss_wumpus": copy.deepcopy(self.kb.poss_wumpus)
        }
        self.history.append(st)
        self.percepts.append(self.env.percept(self.x, self.y, self.wumpus_alive))

    def agir(self):
        percept = self.env.percept(self.x, self.y, self.wumpus_alive)
        self.kb.atualizar(self.x, self.y, percept)

        if percept["glitter"] and not self.has_gold:
            self.has_gold = True
            return "pegou o ouro"

        if self.has_gold and (self.x, self.y) == (1, 1):
            return "saiu da caverna"

        prox = self.kb.proxima_segura()
        if prox:
            self.x, self.y = prox
            return f"moveu para {prox}"
        else:
            return "sem movimentos seguros"


# ---------------------- Interface Pygame ----------------------
pygame.init()
TILE = 100
INFO_HEIGHT = 100
SIZE = 4
WIDTH, HEIGHT = SIZE * TILE, SIZE * TILE + INFO_HEIGHT
FONT = pygame.font.SysFont(None, 26)
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Mundo de Wumpus - Agente Lógico")

# Cores
BG = (240, 240, 240)
GRID = (180, 180, 180)
AGENT = (50, 150, 255)
PIT = (80, 80, 80)
WUMPUS = (200, 50, 50)
GOLD = (255, 215, 0)
SAFE = (180, 255, 180)
VISITED = (200, 200, 255)

def draw_text(txt, x, y, color=(0,0,0)):
    img = FONT.render(txt, True, color)
    screen.blit(img, (x, y))

def draw_world(agente, env, idx):
    st = agente.history[idx]
    screen.fill(BG)

    # desenhar grid
    for i in range(SIZE):
        for j in range(SIZE):
            cx, cy = i * TILE, (SIZE - j - 1) * TILE
            rect = pygame.Rect(cx, cy, TILE, TILE)
            cell = (i+1, j+1)

            color = BG
            if cell in st["visited"]:
                color = VISITED
            elif cell in st["safe"]:
                color = SAFE
            pygame.draw.rect(screen, color, rect)

            pygame.draw.rect(screen, GRID, rect, 2)

            if cell == env.gold:
                pygame.draw.circle(screen, GOLD, rect.center, 15)
            if cell == env.wumpus and st["wumpus_alive"]:
                pygame.draw.circle(screen, WUMPUS, rect.center, 15)
            if cell in env.pits:
                pygame.draw.circle(screen, PIT, rect.center, 10)

            if cell == (st["x"], st["y"]):
                pygame.draw.circle(screen, AGENT, rect.center, 20)

    pygame.draw.rect(screen, GRID, (0, SIZE*TILE, WIDTH, INFO_HEIGHT))
    draw_text(f"Etapa {idx+1}/{len(agente.history)}", 10, SIZE*TILE + 10)
    draw_text(f"Posição: ({st['x']},{st['y']})", 200, SIZE*TILE + 10)
    draw_text(f"Ouro: {'Sim' if st['has_gold'] else 'Não'}", 400, SIZE*TILE + 10)

    pygame.draw.rect(screen, (220,220,220), (WIDTH//4 - 60, SIZE*TILE + 50, 120, 40))
    pygame.draw.rect(screen, (220,220,220), (WIDTH*3//4 - 60, SIZE*TILE + 50, 120, 40))
    draw_text("Anterior", WIDTH//4 - 35, SIZE*TILE + 60)
    draw_text("Próximo", WIDTH*3//4 - 35, SIZE*TILE + 60)

def simular_agente():
    env = Ambiente()
    ag = AgenteLogico(env)
    ag.save_state()

    for _ in range(30):
        acao = ag.agir()
        ag.save_state()
        if acao in ("saiu da caverna", "sem movimentos seguros"):
            break
    return ag, env


def main():
    agente, env = simular_agente()
    idx = 0
    running = True

    while running:
        draw_world(agente, env, idx)
        pygame.display.flip()

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.MOUSEBUTTONDOWN:
                mx, my = event.pos
                if SIZE*TILE + 50 <= my <= SIZE*TILE + 90:
                    if WIDTH//4 - 60 <= mx <= WIDTH//4 + 60:
                        idx = max(0, idx - 1)
                    elif WIDTH*3//4 - 60 <= mx <= WIDTH*3//4 + 60:
                        idx = min(len(agente.history)-1, idx + 1)

    pygame.quit()
    sys.exit()

if __name__ == "__main__":
    main()


pygame 2.6.1 (SDL 2.28.4, Python 3.12.2)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

C:\Users\davi2\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
